<a href="https://colab.research.google.com/github/nestorbalcazar/nestorbalcazar.github.io/blob/master/Rad_RadiosityMethodOpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# ==========================================================================
# Subject:    Heat and mass Transfer
# Topic:      Radiative transfer. Problem 4.
# Author:     Nestor V. Balcazar Arciniega
# Updated:    8 November 2024
# ==========================================================================
# Version:    Python 3.x
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.special import iv, kv
from scipy.integrate import quad

# ==========================================================================
# INPUT PARAMETERS
sigma = 5.67e-8                    # Constante de Stefan-Boltzmann
delta = 1e-15                      # Tolerancia para el criterio de convergencia

eps = np.array([0.9, 1, 0.7, 1])   # Emisividad
T = np.array([250 + 273.15, 25 + 273.15, 50 + 273.15, 25 + 273.15])  # Temperatura en K

# Definir el vector posición de los puntos del contorno cerrado en el sentido de giro del reloj
x = np.array([
    [0, 0],
    [0, 1.5],
    [0.5 + 1.5 * np.cos(65 * (np.pi / 180)), 1.5],
    [0.5, 0]
])

# ==========================================================================
# CÁLCULO DE LOS FACTORES DE FORMA (Fij)
Ni = x.shape[0]  # Número de puntos en el contorno
F = np.zeros((Ni, Ni))

for i in range(Ni):
    p1_i = x[i]
    p2_i = x[(i + 1) % Ni]
    Li = np.linalg.norm(p1_i - p2_i)
    for j in range(Ni):
        if i != j:
            p1_j = x[j]
            p2_j = x[(j + 1) % Ni]
            d4 = np.linalg.norm(p1_i - p2_j)
            d3 = np.linalg.norm(p2_i - p1_j)
            d2 = np.linalg.norm(p1_i - p1_j)
            d1 = np.linalg.norm(p2_i - p2_j)
            F[i, j] = ((d1 + d2) - (d3 + d4)) / (2 * Li)

print("Matriz de Factores de Forma (Fij):")
print(F)

# ==========================================================================
# CÁLCULO ITERATIVO DE RADIOSIDAD j
j = eps * sigma * (T ** 4)   # Condición inicial para j (radiosidad)
j0 = j.copy()                 # Copia inicial para comparación en la iteración

while True:
    RHS = (1 - eps) * F @ j  # Producto matricial para el cálculo de RHS en cada i
    j = (RHS + eps * sigma * T ** 4) / (1 - (1 - eps) * np.diag(F))

    # Cálculo del error y verificación de convergencia
    error = np.linalg.norm(j - j0)
    if error < delta:
        break

    # Actualizar j0 para la siguiente iteración
    j0 = j.copy()

# ==========================================================================
# CÁLCULO DE g, q Y res_qradS
g = np.sum(F * j, axis=1)      # Suma en cada fila para calcular g
q = j - g                      # Diferencia entre j y g

# Longitudes de cada segmento en el contorno
S = np.array([np.linalg.norm(x[i] - x[(i + 1) % Ni]) for i in range(Ni)])

# Producto escalar q * S
res_qradS = np.sum(q * S)

# Mostrar resultados finales
print("S =", S)
print("j =", j)
print("g =", g)
print("q =", q)
print("res_qradS = sum(q * S) =", res_qradS)


Matriz de Factores de Forma (Fij):
[[0.         0.2511858  0.60919381 0.13962039]
 [0.33227762 0.         0.52086361 0.14685877]
 [0.56113987 0.36268838 0.         0.07617175]
 [0.41886117 0.33305437 0.24808446 0.        ]]
S = [1.5        1.13392739 1.62845446 0.5       ]
j = [3941.87501068  681.21957098  654.0047731   661.83006504]
g = [ 661.93331413 1747.63968826 2509.42639224 2040.22995458]
q = [ 3279.94169654 -1066.42011728 -1855.42161914 -1378.39988954]
res_qradS = sum(q * S) = -2.7284841053187847e-12
